# 해당쿼리로 데이터 추출

SELECT datetime, shot_no, tool_number, current_spindle, current_x, current_z, vibration, rpm, rpm_set, feed, feed_set, load_1, servo_load_x, servo_load_z, servo_current_x, servo_current_z from TPOP_MACHINE_PARAMETER
WHERE mc_id = 22 and date between '2022-07-13' AND '2022-10-12'

- 날짜 기준 범위는 1개월 반씩 분리해서 Query 수행

In [1]:
!jupyter nbconvert --to python module.py

[NbConvertApp] Converting notebook module.py to python
Traceback (most recent call last):
  File "C:\Users\jhpark\anaconda3\lib\site-packages\nbformat\reader.py", line 14, in parse_json
    nb_dict = json.loads(s, **kwargs)
  File "C:\Users\jhpark\anaconda3\lib\json\__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "C:\Users\jhpark\anaconda3\lib\json\decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "C:\Users\jhpark\anaconda3\lib\json\decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\jhpark\anaconda3\Scripts\jupyter-nbconvert-script.py", line 10, in <module>
    sys.exit(main())
  File "C:\Users\jhpark\anaconda3\lib\site-packages\jupyter_core\application.py", line 270,

# 준비

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from sklearn.ensemble import IsolationForest
warnings.filterwarnings('ignore')
import joblib

In [4]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', 200)

# 함수 정의

## 일반

### 컬럼 데이터 형변환 (64bit → 32bit)
 * 데이터 용량 줄이기 위한 용도

In [5]:
#주어진 dataframe에서 데이터 타입이 64bit인 경우 32bit로 변경하여 데이터 용량 축소
def change_data_type_64bit_to_32bit(data):
    column_names = list(data.columns.values)
    
    for col in column_names:
        data_type = str(data[col].dtype)
        
        if data_type == 'int64':
            data[col] = data[col].astype('int32')
        elif data_type == 'float64':
            data[col] = data[col].astype('float32')
        #LJY_20220929 : 컬럼의 data type이 'object'인데 실제 값은 numeric인 경우 int32로 변환
        # 추후 데이터 타입 판별로 변경 필요 (numeric인지 체크하는 api에서 datetime을 걸러내지 못해서 임시로 column name으로 체크하도록 처리)
        else:
            if col != "datetime":
                data[col] = data[col].replace('\\0', 0)
                data[col] = data[col].fillna(0).astype('int32')
#                data[col] = (data[col].fillna(0).astype('int32').astype(object).where(data[col].notnull()))
#                data[col] = data[col].notnull().astype('int32')
    
    column_names.clear()
    del column_names

### tool 정보 추출
 * 'tool_number' 필드에서 공구 정보를 추출하여 별도의 컬럼으로 구성

In [6]:
# tool 번호 및 상태 생성
def extract_tool_info_from_data(data):
    if 'tool_number' not in data.columns:
        return
    
    data['tool_state'] = data['tool_number']%100
    data['tool'] = (data['tool_number'] - data['tool_state'])/100
    if data['tool'].isnull().sum() > 0:
        data = data.dropna(axis = 0)
    data['tool'] = data['tool'].astype(np.int32)

## 데이터 불러오기

### 주어진 경로의 csv 파일 목록 찾기

In [7]:
#주어진 경로의 .csv파일을 찾아서 list 형태로 반환
def find_csv_files(path):
    file_list = os.listdir(path)
    
    file_list_csv = [file_csv for file_csv in file_list if file_csv.endswith('.csv')]
    file_list_csv
    
    file_list.clear()
    del file_list
    
    return file_list_csv

### 주어진 경로의 대상 csv파일을 불러오기
 * 입력인자 중 sensor_data_only : True이면 전류 센서 데이터만 포함, False이면 Focas 데이터를 포함한 파일을 의미
 * 데이터 종류를 입력으로 하여 해당하는 데이터 타입의 .csv파일 Load
 * 데이터 타입 형변환 (64bit → 32bit, object타입의 경우 datetime 컬럼을 제외하고 int32로 변환)

In [8]:
#주어진 경로의 .csv파일을 찾아서 모두 병합한 dataframe 구성
def build_dataframe_from_cvs_files(path, sensor_data_only=True, display_report=False):
    file_list_csv = find_csv_files(path)
    
    df = pd.DataFrame()
    
    if len(file_list_csv) < 1:
        return df

    #전류 센서 & focas 데이터 포함 파일의 header부분(컬럼명 나열된 line) 정보
    all_data_columns = '"datetime","shot_no","tool_number","current_spindle","current_x","current_z","vibration","rpm","rpm_set","feed","feed_set","load_1","servo_load_x","servo_load_z","servo_current_x","servo_current_z"'
    # all_data_columns = ',datetime,tool_number,shot_no,current_x,current_z,current_spindle,rpm,feed,load_1,servo_load_x,servo_load_z,servo_current_x,servo_current_z'

    
    #전류 센서데이터만 포함한 파일의 header부분(컬럼명 나열된 line) 정보
    sensor_data_only_columns = '"datetime","tool_number","shot_no","current_spindle","current_x","current_z"'
    
    for csv_idx in file_list_csv:
        first_line = ""
        
        #encoding을 utf-8-sig로 해야 파일에서 읽은 문자열의 맨 앞에 '\ufeff'가 붙는 현상을 막을 수 있음
        # - utf-8로 해서 읽으면 맨 앞에 '\ufeff'가 붙어서 위에 정의한 all_data_columns, sensor_data_only_columns와 비교 시 무조건 False 발생
        with open(csv_idx, encoding="utf-8-sig") as f:
            first_line = f.readline()
            print(first_line)
            first_line = first_line.strip('\n')
            f.close()
        
        if display_report is True:
            print(first_line)
        
        #전류 센서 & focas 데이터 포함 파일 (무의미한 column은 DB에서 조회할 때 제외하고 조회 완료)
        if first_line == all_data_columns:
            if display_report is True:
                print("file {} : all data".format(csv_idx))
                
            if sensor_data_only is True:
                continue
        #전류 센서 데이터만 포함한 파일
        elif first_line == sensor_data_only_columns:
            if display_report is True:
                print("file {} : current sensor data only".format(csv_idx))
                
            if sensor_data_only is False:
                continue
        else:
            if display_report is True:
                print("file {} : invalid data".format(csv_idx))
                
            continue
            
        csv_df = pd.read_csv(csv_idx)
            
        #데이터 타입을 64bit → 32bit로 변경하여 용량 축소
        change_data_type_64bit_to_32bit(csv_df)
    
        #print(csv_df.info())

        df = pd.concat([df, csv_df], axis = 0)
        del csv_df
    
    file_list_csv.clear()
    del file_list_csv
    
    return df

### 주어진 경로의 대상 파일을 불러오기
 * 입력인자 중 sensor_data_only : True이면 전류 센서 데이터만 포함, False이면 Focas 데이터를 포함한 파일을 의미
 * current directory에서 .parquet파일을 찾아서 불러오기 
 * .parquet파일이 없는 경우 .csv를 읽어서 통합한 후 .parquet로 저장

In [9]:
#--------------------------------------------------------------
#☆☆ Parquet(파케이)란?
#   - Apache Parquet은 쿼리 속도를 높이기 위한 최적화를 제공하는 열 형식 파일 형식이며 CSV 또는 JSON보다 훨씬 효율적인 파일 형식입니다.
#   - <참고> : https://pearlluck.tistory.com/561
#--------------------------------------------------------------
#
#
#Parquet(파케이) 파일이 있는 경우 해당 파일을 읽고, 그렇지 않은 경우 .csv 파일을 순회하면서 읽어서 merge
#pyarrow 모듈 설치해야 .parquet 파일 입출력 기능 사용 가능
# !pip install pyarrow

def read_data(sensor_data_only=True, extract_tool_info=True, display_report=False):
    df = pd.DataFrame()
    
    #전류 센서로만 구성된 데이터를 사용하려고 하는 경우
    if sensor_data_only is True:
        if os.path.isfile('./VL04_data_sensor_only.parquet'): #파일이 있는 경우
            df = pd.read_parquet('./VL04_data_sensor_only.parquet')  #.parquet 데이터 ()
    #focas 데이터까지 포함된 데이터를 사용하려고 하는 경우
    else:    
        if os.path.isfile('./VL04_data.parquet'): #파일이 있는 경우
            df = pd.read_parquet('./VL04_data.parquet')  #.parquet 데이터 ()
    
    if df.empty:
        #현재 디렉토리의 .csv파일을 모두 읽어서 하나의 dataframe으로 구성
        df = build_dataframe_from_cvs_files('./', sensor_data_only, display_report)

        df = df.reset_index(drop=True)

        #dataframe을 .pqrquet 파일로 저장하여 추후 상대적으로 적은 메모리의 파일을 빠르게 읽어서 사용 할 수 있도록 처리
        df.to_parquet('./VL04_data.parquet', compression='gzip') #압축 파일 형식(gzip, snappy, ..)을 지정하여 pqrquet 파일 저장
        
    # tool 번호 및 상태 생성
    if not df.empty and extract_tool_info:
        extract_tool_info_from_data(df)
        
    return df

## 전처리 : 유효한 shot 정보 도출
 * 'shot_no' 필드 값을 기준으로 shot 구간 찾기
 * shot 구간 내부의 공구 사용 순서 및 각 공구 별 데이터 index 구간 도출
 * 공구 사용 순서에서 유효한 공구 사용 패턴 구간 찾기
 * 유효한 공구 사용 패턴 구간에 대해 numbering하여 별도의 'real_shot' 필드에 값 설정 (디폴트 : -1) 

### dataframe에 사용된 공구 순서 도출

In [10]:
#dataframe에 사용된 공구를 순서대로 추출해서 list형태로 반환
def extract_tool_list(data, report_result=False):
    import itertools

    tool_list = []
    #print([(k, (g)) for k, g in itertools.groupby(df_shot_specific['tool'])])
    for k, g in itertools.groupby(data['tool']):
        tool_list.append(k)

    if report_result is True:
        print(tool_list)
    
    return tool_list

In [11]:
#dataframe에 사용된 공구를 순서대로 추출해서 list형태로 반환
def extract_tool_list_with_range(data, report_result=False):
    import itertools

    irow = 0
    tool_list = []
    range_list_of_tool = []
    
    #print([(k, (g)) for k, g in itertools.groupby(df_shot_specific['tool'])])
    for k, g in itertools.groupby(data['tool']):
        tool_list.append(k)
        nrow = len(list(g))
        range_list_of_tool.append((irow,irow+nrow-1))
        irow += nrow

    if report_result is True:
        print(tool_list)
        print(range_list_of_tool)
    
    return tool_list, range_list_of_tool

In [12]:
#dataframe에서 (1) 주어진 shot_no에 해당하는 데이터를 추출한 후, (2) 해당 데이터에 사용된 공구를 순서대로 가져와서 반환
def extract_tool_list_of_specific_shot(data, shot_no, report_result=False):
    df_shot = data[data['shot_no'] == shot_no]
    
    if report_result is True:
        print("===== shot{} ============================\n\n(1) dataframe -----------------\n".format(shot_no))
        print(df_shot)
        print("(2) used tool -----------------\n")
        print(df_shot['tool'].unique())
        print("(3) extract order using tools -----------------\n")
    
    tool_list = extract_tool_list(df_shot, report_result)
    
    del df_shot
    
    return tool_list

In [13]:
#dataframe에서 전체 shot에 대해 아래의 과정 수행
#   - 각 shot 별로 사용된 공구 순서를 추출
#   - shot & 공구 사용 순서정보를 column으로 하는 dataframe을 구성한 후 반환
def extract_tool_list_of_all_shots(data, report_result=False):
    column_names = ['shot_no', 'order_using_tool']
    df_by_shot = pd.DataFrame(columns=column_names)
    row = 0

    for shot in df['shot_no'].unique():
        tool_list = extract_tool_list_of_specific_shot(df, shot, report_result)
        str_tool_list = ' '.join(map(str, tool_list))
        df_by_shot.loc[row] = [shot, str_tool_list]
        
        del tool_list
        
        row += 1

    print(df_by_shot)
    
    return df_by_shot

### 유효 shot 구간 도출
 * 정상 공구 사용 패턴을 가지는 구간을 하나의 유효 shot으로 판별하고 별도의 column('real_shot')에 해당 numbering

In [14]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

In [15]:
#dataframe에서 (1) 주어진 shot_no에 해당하는 데이터를 추출한 후
#(2) 해당 데이터에 사용된 공구 순서 및 각 공구 별 row index 범위를 순서대로 가져옴
#(3) 정상 패턴 가공이 이루어지는 구간을 도출 (유효한 shot으로 판단할 수 있는 구간)
#(4) (3)에서 찾은 구간 데이터의 'real_shot' 컬럼에 새로 shot numbering
def find_and_mark_valid_shot_of_specific_shot(data, shot_no, real_shot_no, report_result=False):
    df_shot = data[data['shot_no'] == shot_no]
    
    if report_result is True:
        print("===== shot{} ============================\n\n(1) dataframe -----------------\n".format(shot_no))
        print(df_shot)
        print("(2) used tool -----------------\n")
        print(df_shot['tool'].unique())
        print("(3) extract order using tools -----------------\n")
    
    tool_list, range_list_of_tool = extract_tool_list_with_range(df_shot, report_result=False)

    tool_pattern = [1, 5, 9, 11, 7, 3, 11, 7]
    sub_list = find_sub_list(tool_pattern, tool_list)
    
    if len(sub_list) < 1:
        if report_result is True:
            print("shot_no {} of raw data : invalid_shot".format(shot_no))

        return 0
    
    if report_result is True:
        print(sub_list)
    
    start_index = df_shot.index[0]
    count = 0
    
    if report_result is True: 
        if sub_list[0][0] > 0:
            print("------ invalid data\n")
            print(data.iloc[start_index: start_index+range_list_of_tool[sub_list[0][0]][0]])

    for sub in sub_list:
        real_shot_no += 1
        sub_start = start_index + range_list_of_tool[sub[0]][0]
        sub_end   = start_index + range_list_of_tool[sub[1]][1]
        print("sub({}:{}) - real_shot_no {}".format(sub_start, sub_end, real_shot_no))
        
        #========================================================================
        #LJY_20220926 : iloc 함수를 사용하여 구간 access 후 값 설정 시 pandas ver.1.3.5 이후로(1.4.0부터) 동작하지 않던 오류 수정
        #------------------------------------------------------------
        #오류 : 기존에 작성된 코드
        #data.iloc[sub_start:sub_end]['real_shot'] = real_shot_no
        
        #------------------------------------------------------------
        #방법1 : column을 특정한 후 row index range 지정하여 값 설정
        data.real_shot.iloc[sub_start:sub_end] = real_shot_no
        
        #------------------------------------------------------------
        #방법2 : row index range와 column index를 지정하여 값 설정
        # print('real_shot_no : ',real_shot_no)
        # data.iloc[sub_start:sub_end+1, data.columns.get_loc('real_shot')] = real_shot_no
        #========================================================================
        
        if report_result is True:
            print("------ valid shot[{}]\n".format(count))
            print(data.iloc[sub_start:sub_end+1])
            
        count += 1
    
    tool_list.clear()
    range_list_of_tool.clear()
    sub_list.clear()
    
    del df_shot
    
    return count

In [16]:
#dataframe에서 유효한 공구 사용 패턴을 가지는 구간을 찾아서 별도의 컬럼('real_shot')에 새로 number 부여
# - 유효하지 않은 구간의 경우 'real_shot' 값을 -1로 설정
def find_and_mark_valid_shot(data, report_result=False):
    real_shot_no = 0
    
    data['real_shot'] = -1
    
    for shot in data['shot_no'].unique():
        real_shot_no += find_and_mark_valid_shot_of_specific_shot(data, shot, real_shot_no, report_result)
        print(real_shot_no)

### 유효 shot 구간 내의 시작 부위의 유휴시간 데이터 삭제
 * 정상 공구 사용 패턴을 가지는 구간을 하나의 유효 shot으로 판별하고 별도의 column('real_shot')에 해당 numbering

In [17]:
#dataframe의 시작 부위의 idle section에 대한 'real_shot' 정보 초기화(값: -1)
# => 실제 가공이 이루어진 유효한 구간을 찾아냄
def remove_idle_section_at_the_start_of_valid_shot(data):
    #print(data)
    
    #스핀들 전류값이 0보다 크면 가공이 이루어진 구간으로 판단하여 아무 처리하지 않고 return
    if data.iloc[0]['current_spindle'] > 0:
        return
    
    last_index_of_idle_section = data[data['current_spindle'] > 0].index[0]-1
    
    print("index : {}~{}\n".format(data.index[0], last_index_of_idle_section))
    #print(data.loc[data.index[0]:last_index_of_idle_section])
    data.loc[data.index[0]:last_index_of_idle_section, 'real_shot'] = -1 #idle 구간의 valid shot 정보 초기화
    #print(data.loc[data.index[0]:last_index_of_idle_section])
    return

In [18]:
#dataframe의 유효한 shot 구간('real_shot' != -1인 구간) 별로 시작 부위의 idle section에 대한 'real_shot' 정보 초기화(값: -1)
# => 실제 가공이 이루어진 유효한 구간을 찾아냄
def remove_idle_sections_at_the_start_of_valid_shots(data):
    for shot in data['real_shot'].unique():
        if shot < 0:
            continue
            
#        df_by_shot = data[data['real_shot'] == shot]
        
#        if df_by_shot.iloc[0]['shot_no'] == 6517:
#            print(df_by_shot)
            
#        print(df_by_shot)
        index = data[data['real_shot'] == shot].index
    
        remove_idle_section_at_the_start_of_valid_shot(data.loc[index[0]:index[0]+len(index)-1])
        
        print(data.loc[index[0]:index[0]+len(index)-1])

    return

## 시각화

In [19]:
#dataframe에서 컬럼을 다중선택하여 데이터를 차트로 가시화
#① data : dataframe 입력
#② column_names : list 형태의 column명을 입력   ex) ['col1'], ['col1', 'col2'] 
#③ dataframe에서 특정 범위만을 선택해서 차트를 가시화 하려면 index_min 또는 index_max를 지정
#   .. index_min : 디폴트값(-1)인 경우에는 데이터의 시작 row부터 포함됨
#   ..  index_max : 디폴트값(-1)인 경우에는 데이터의 끝 row까지 포함됨
#④ 차트 제목 표시 관련 설정
#   .. title : 차트의 제목 문자열
#   .. title_font_size : 차트의 제목을 표시할 font size
#⑤ 차트 크기 설정
#   .. figsize_horz, figsize_vert : 차트의 가로, 세로 크기
#⑥ 범례 표시 관련 설정
#   .. legend_font_size : 범례를 표시할 font size
#   .. legend_location : 범례 표시 위치 ("upper right", "lower right", "upper left", "lower left")
#⑦ 그래프 표현 관련 설정 : 선 or marker 표시
#   .. linestyle : 선 스타일의 이름을 "solid", "dashed", "dotted", "dashdot"와 같은 형식으로 입력하거나 아래를 참고하여 입력
'''
'-'  solid line style
'--' line style
'-.' dash-dot line style
':'  dotted line style
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
's'  square marker
'p'  pentagon marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
#  .. marker : 아래를 참고하여 입력 ( https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html)
'''
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
'8'  octagon marker
's'  square marker
'p'  pentagon marker
'P'  plus (filled) marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'X'  x (filled) marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
def show_plot(data, column_names, index_min = -1, index_max = -1, title = None, title_font_size=20,
              x_axis_title = None, y_axis_title = None, axes_title_font_size=18, tick_label_font_size=15,
              figsize_horz=20, figsize_vert=10, legend_font_size=15, legend_location="upper right",
              linestyle = 'none', marker='.', marker_size=10):
    
    parameters = { 'figure.titlesize':title_font_size, 'axes.titlesize': axes_title_font_size,
                  'axes.labelsize' : tick_label_font_size, 'legend.fontsize' : legend_font_size }
    plt.rcParams.update(parameters)
    
    plt.figure(figsize = (figsize_horz, figsize_vert))
    
    str_col_names = '{}'.format(','.join(column_names))#(f"'{col}'" for col in column_name))
    print(str_col_names)
    
    if index_min > -1 and index_max > -1 and index_min < index_max:
        for column in list(column_names):
            plt.plot(data.index[index_min:index_max], data.iloc[index_min:index_max][column], marker=marker, linestyle=linestyle, markersize=marker_size)
    elif index_min > -1:
        for column in list(column_names):
            plt.plot(data.index[index_min:], data.iloc[index_min:][column], marker=marker, linestyle=linestyle, markersize=marker_size)
    elif index_max > -1:
        for column in list(column_names):
            plt.plot(data.index[:index_max], data.iloc[:index_max][column], marker=marker, linestyle=linestyles, markersize=marker_size)
    else:
        #str_col_names = ','.join('{0}'.format(col) for col in column_name)
        for column in list(column_names):
            plt.plot(data.index, data[column], marker=marker, linestyle=linestyle, markersize=marker_size)
    
    if title is not None:
        plt.title(title)

    plt.legend(column_names, loc = legend_location)
    plt.xticks(fontsize = tick_label_font_size)
    plt.yticks(fontsize = tick_label_font_size)
    
    if x_axis_title is not None:
        plt.xlabel(x_axis_title)
    if y_axis_title is not None:
        plt.ylabel(y_axis_title)


In [20]:
#dataframe에서 컬럼을 다중선택하여 데이터를 차트로 가시화
#① data : dataframe 입력
#② target_column_name : 값을 관찰할 컬럼
#③ condition_column : 조건 판별 대상 컬럼명  ex) 'state'
#④ condition_values : 일치 여부 대상 value 목록  ex) [1], [0, 1]
#⑤ dataframe에서 특정 범위만을 선택해서 차트를 가시화 하려면 index_min 또는 index_max를 지정
#   .. index_min : 디폴트값(-1)인 경우에는 데이터의 시작 row부터 포함됨
#   ..  index_max : 디폴트값(-1)인 경우에는 데이터의 끝 row까지 포함됨
#⑥ 차트 제목 표시 관련 설정
#   .. title : 차트의 제목 문자열
#   .. title_font_size : 차트의 제목을 표시할 font size
#⑦ 차트 크기 설정
#   .. figsize_horz, figsize_vert : 차트의 가로, 세로 크기
#⑧ 범례 표시 관련 설정
#   .. legend_font_size : 범례를 표시할 font size
#   .. legend_location : 범례 표시 위치 ("upper right", "lower right", "upper left", "lower left")
#⑨ 그래프 표현 관련 설정 : 선 or marker 표시
#   .. linestyle : 선 스타일의 이름을 "solid", "dashed", "dotted", "dashdot"와 같은 형식으로 입력하거나 아래를 참고하여 입력
'''
'-'  solid line style
'--' line style
'-.' dash-dot line style
':'  dotted line style
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
's'  square marker
'p'  pentagon marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
#  .. marker : 아래를 참고하여 입력 ( https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html)
'''
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
'8'  octagon marker
's'  square marker
'p'  pentagon marker
'P'  plus (filled) marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'X'  x (filled) marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
def show_plot_comparing_data_by_condition(data, target_column_name, condition_column, condition_values, data_names = ('condition is true','condition is false'),
                                           index_min = -1, index_max = -1, title = None, title_font_size=20,
                                           x_axis_title = None, y_axis_title = None, axes_title_font_size=18,
                                           tick_label_font_size=15, figsize_horz=20, figsize_vert=10,
                                           legend_font_size=15, legend_location="lower right", linestyle = 'none',
                                           marker='.', marker_size=10):
    
    parameters = { 'figure.titlesize':title_font_size, 'axes.titlesize': axes_title_font_size,
                  'axes.labelsize' : tick_label_font_size, 'legend.fontsize' : legend_font_size }
    plt.rcParams.update(parameters)
    
    plt.figure(figsize = (figsize_horz, figsize_vert))
    
    condition_true_data = pd.DataFrame()
    condition_false_data = pd.DataFrame()
      
    if index_min > -1 and index_max > -1 and index_min < index_max:
        condition_true_data = data.iloc[index_min:index_max].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[index_min:index_max].query(~data[condition_column].isin(condition_values))
    elif index_min > -1:
        condition_true_data = data.iloc[index_min:].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[index_min:].query(~data[condition_column].isin(condition_values))        
    elif index_max > -1:
        condition_true_data = data.iloc[:index_max].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[:index_max].query(~data[condition_column].isin(condition_values))        
    else:
        condition_true_data = data[data[condition_column].isin(condition_values)]
        condition_false_data = data[~data[condition_column].isin(condition_values)]
        
    plt.plot(condition_true_data.index, condition_true_data[target_column_name], marker=marker, linestyle=linestyle, label=data_names[0], color='red', markersize=marker_size)
    plt.plot(condition_false_data.index, condition_false_data[target_column_name], marker=marker, linestyle=linestyle, label=data_names[1], markersize=marker_size)
    
    if title is not None:
        plt.title(title)

    plt.legend(data_names, loc = legend_location)
    plt.xticks(fontsize = tick_label_font_size)
    plt.yticks(fontsize = tick_label_font_size)
    
    if x_axis_title is not None:
        plt.xlabel(x_axis_title, fontsize = axes_title_font_size)
    if y_axis_title is not None:
        plt.ylabel(y_axis_title, fontsize = axes_title_font_size)

## 모델 수립

### 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여 dataframe 구성

In [ ]:
def delete_outliers(data, column):
    Q1 = np.percentile(data[column], 25)
    Q3 = np.percentile(data[column], 75)
    IQR = Q3 - Q1

    outlier_step = 1.5 * IQR
    
    print('Q1 = {}, Q3 = {}, IQR = {}, outlier_step = {}, range = ({}. {})'.format(Q1, Q3, IQR, outlier_step, Q1-outlier_step, Q3+outlier_step))

    outlier_index = data[(data[column] < Q1) | (data[column] > Q3 + outlier_step)].index
        
    print(outlier_index)
    
    data.drop(outlier_index, inplace=True)

In [21]:
# 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여  dataframe으로 구성
def build_representative_data_for_tool_machining(data, tool_no, target_column_name, apply_robust_scaler=False, except_min_value=False, except_max_value=False, delete_outlier=False,
                                                 delete_zero_value=False, except_tool_cancel_state=False, except_tool_end_state=False):
    if target_column_name not in data.columns:
        print("{} is not in columns\nPlease input valid column name".format(target_column_name))
        return
    
    condition = (data['tool']==tool_no) & (data['real_shot'] != -1)
    
    #LJY_20221005 : tool취소 상태 데이터 제거 옵션 추가
    if except_tool_cancel_state is True:
        condition = condition & (data['tool_state'] != 0)
        
    if except_tool_end_state is True:
        condition = condition & (data['tool_state'] != 9)
        
    #data_tool = data[(data['tool']==tool_no) & (data['real_shot'] != -1)]
    data_tool = data[condition]
    
    if delete_zero_value is True:
        data_tool.drop(data_tool[data_tool[target_column_name] == 0].index, inplace=True)
        
    if delete_outlier is True:
        print('before deleting outliers : {}'.format(len(data_tool.index)))
        delete_outliers(data_tool, target_column_name)
        print('after deleting outliers : {}'.format(len(data_tool.index)))
    
    if except_min_value is True:
        min_value = data_tool[target_column_name].min()
        data_tool.drop(data_tool[data_tool[target_column_name] == min_value].index, inplace=True)
        
    if except_max_value is True:
        max_value = data_tool[target_column_name].max()
        data_tool.drop(data_tool[data_tool[target_column_name] == max_value].index, inplace=True)
    
    if apply_robust_scaler is True:
        from sklearn import metrics
        #from sklearn.preprocessing import MinMaxScaler
        #from sklearn.preprocessing import StandardScaler
        from sklearn.preprocessing import RobustScaler
        
        rs = RobustScaler()
        #ms = MinMaxScaler()
        data_tool[[target_column_name]] = rs.fit_transform(data_tool[[target_column_name]])
    
    #real shot no.
#    data_shot_no = data_tool['real_shot'].to_frame(name='real_shot')
    group_by_shot = data_tool.groupby(['real_shot'])
    #print(group_by_shot.groups.keys())
    
    #표준편차
    data_std = (group_by_shot.std()[target_column_name].to_frame(name='std'))
    #표준오차
    data_sem = (group_by_shot.sem()[target_column_name].to_frame(name='sem'))
    #합
    data_sum = (group_by_shot.sum()[target_column_name].to_frame(name='sum'))
    #평균
    data_mean = (group_by_shot.mean()[target_column_name].to_frame(name='mean'))
    #중앙값
    data_median = (group_by_shot.median()[target_column_name].to_frame(name='median'))
    #분산
    data_var = (group_by_shot.var()[target_column_name].to_frame(name='var'))
    #최대값
    data_max = (group_by_shot.max()[target_column_name].to_frame(name='max')) 
    #0.25분위수
    data_quantile_1_per_4 = (group_by_shot.quantile(0.25)[target_column_name].to_frame(name='quantile(0.25)')) 
    #0.75분위수
    data_quantile_3_per_4 = (group_by_shot.quantile(0.75)[target_column_name].to_frame(name='quantile(0.75)')) 
    #왜도
    data_skew = (group_by_shot.skew()[target_column_name].to_frame(name='skew')) 

    data_representative = pd.concat([data_mean, data_std, data_median, data_max, data_var, data_sem, data_sum, data_quantile_1_per_4, data_quantile_3_per_4, data_skew],axis=1)
    
    data_representative.set_index(pd.Series(group_by_shot.groups.keys()))
    
    return data_representative

In [ ]:
# 다변량 처리를 위한 함수 정의
# 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여  dataframe으로 구성
# 유효한 shot을 기준으로 주어진 공구에 대한 주어진 field의 대푯값을 계산하여  dataframe으로 구성
def build_representative_multi_data_for_tool_machining(data, tool_no, stat='mean', apply_robust_scaler=False, except_min_value=False, except_max_value=False, delete_outlier=False,
                                                 delete_zero_value=False, except_tool_cancel_state=False, except_tool_end_state=False, *target_column_name):
    # stat = std, sem, sum, mean, median, var, max, 25%, 75%, skew, overall
    target_column_name = list(target_column_name)
    for column_idx in target_column_name:
        if column_idx not in data.columns:
            print("{} is not in columns\n Please input valid column name".format(column_idx))
            return
    condition = (data['tool']==tool_no) & (data['real_shot'] != -1)
    
    #LJY_20221005 : tool취소 상태 데이터 제거 옵션 추가
    if except_tool_cancel_state is True:
        condition = condition & (data['tool_state'] != 0)
        
    if except_tool_end_state is True:
        condition = condition & (data['tool_state'] != 9)
        
    #data_tool = data[(data['tool']==tool_no) & (data['real_shot'] != -1)]
    data_tool = data[condition]
    
    if delete_zero_value is True:
        zero_index = []
        for column_idx2 in target_column_name:
            zero_index.append(data_tool[data_tool[column_idx2] == 0].index)
        zero_result = list(set(zero_index))
        print(zero_result)
        data_tool.drop(data_tool[data_tool[column_idx2] == 0].index, inplace=True)
        
    if delete_outlier is True:
        print('before deleting outliers : {}'.format(len(data_tool.index)))
        for column_idx3 in target_column_name:
            delete_outliers(data_tool, column_idx3)
        print('after deleting outliers : {}'.format(len(data_tool.index)))
    
    if except_min_value is True:
        for column_idx4 in target_column_name:
            globals()["{}_min_value".format(column_idx4)] = data_tool[column_idx4].min()
            data_tool.drop(data_tool[data_tool[column_idx4] == globals["{}_min_value".format(column_idx4)]].index, inplace=True)
        
    if except_max_value is True:
        for column_idx5 in target_column_name:
            globals()["{}_max_value".format(column_idx5)] = data_tool[column_idx5].max()
            data_tool.drop(data_tool[data_tool[column_idx5] == globals()["{}_max_value".format(column_idx5)]].index, inplace=True)
    
    if apply_robust_scaler is True:
        from sklearn import metrics
        #from sklearn.preprocessing import MinMaxScaler
        #from sklearn.preprocessing import StandardScaler
        from sklearn.preprocessing import RobustScaler
        
        rs = RobustScaler()
        #ms = MinMaxScaler()
        data_info = data_tool[['datetime', 'shot_no', 'tool_number', 'tool_state', 'tool', 'real_shot']]
        data_tool = data_tool.drop(data_info.columns, axis = 1)
        data_tool[target_column_name] = rs.fit_transform(data_tool[target_column_name])
        data_tool = pd.concat([data_info, data_tool], axis = 1)
    
    
    #real shot no.
#    data_shot_no = data_tool['real_shot'].to_frame(name='real_shot')
    group_by_shot = data_tool.groupby(['real_shot'])
#    print(group_by_shot.groups.keys())
    
    #표준편차
    # data_std = pd.DataFrame(group_by_shot.std()[target_column_name], columns = ['std_' + str(i) for i in target_column_name])
    
    stat_list = ['std', 'sem', 'sum', 'mean', 'median', 'var', 'max', '25%', '75%', 'skew', 'overall']
#     try : 
#         stat in stat_list == True
#     except ValueError as val:
#         print('대표값 목록에 있는 값으로 입력해주세요!')
#         print('대표값 목록은 {}입니다.'.format(','.join()))
#     else:
    if stat == 'std':
        data_representative = pd.DataFrame(group_by_shot.std()[target_column_name])
        data_representative.columns = ['std_' + str(i) for i in target_column_name]
    elif stat == 'sem':
        data_representative = pd.DataFrame(group_by_shot.sem()[target_column_name])
        data_representative.columns = ['sem_' + str(i) for i in target_column_name]
    elif stat == 'sum':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['sum_' + str(i) for i in target_column_name]
    elif stat == 'mean':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['mean_' + str(i) for i in target_column_name]
    elif stat == 'median':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['median_' + str(i) for i in target_column_name]
    elif stat == 'var':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['var_' + str(i) for i in target_column_name]
    elif stat == 'max':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['max_' + str(i) for i in target_column_name]
    elif stat == '25%':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['q1(25%)_' + str(i) for i in target_column_name]
    elif stat == '75%':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['q3(75%)_' + str(i) for i in target_column_name]
    elif stat == 'skew':
        data_representative = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_representative.columns = ['skew_' + str(i) for i in target_column_name]
    elif stat == 'overall' : 
        data_std = pd.DataFrame(group_by_shot.std()[target_column_name])
        data_std.columns = ['std_' + str(i) for i in target_column_name]
        #표준오차
        data_sem = pd.DataFrame(group_by_shot.sem()[target_column_name])
        data_sem.columns = ['sem_' + str(i) for i in target_column_name]
        #합
        data_sum = pd.DataFrame(group_by_shot.sum()[target_column_name])
        data_sum.columns = ['sum_' + str(i) for i in target_column_name]
        #평균
        data_mean = pd.DataFrame(group_by_shot.mean()[target_column_name])
        data_mean.columns = ['mean_' + str(i) for i in target_column_name]
        #중앙값
        data_median = pd.DataFrame(group_by_shot.median()[target_column_name])
        data_median.columns = ['median_' + str(i) for i in target_column_name]
        #분산
        data_var = pd.DataFrame(group_by_shot.var()[target_column_name])
        data_var.columns = ['var_' + str(i) for i in target_column_name]
        #최대값
        data_max = pd.DataFrame(group_by_shot.max()[target_column_name])
        data_max.columns = ['max_' + str(i) for i in target_column_name]
        #0.25분위수
        data_quantile_1_per_4 = pd.DataFrame(group_by_shot.quantile(0.25)[target_column_name])
        data_quantile_1_per_4.columns = ['q1_' + str(i) for i in target_column_name]
        #0.75분위수
        data_quantile_3_per_4 = pd.DataFrame(group_by_shot.quantile(0.75)[target_column_name])
        data_quantile_3_per_4.columns = ['q3_' + str(i) for i in target_column_name]
        #왜도
        data_skew = pd.DataFrame(group_by_shot.skew()[target_column_name]) 
        data_skew.columns = ['skew_' + str(i) for i in target_column_name]

        data_representative = pd.concat([data_mean, data_std, data_median, data_max, data_var, data_sem, data_sum, data_quantile_1_per_4, data_quantile_3_per_4, data_skew],axis=1)

        data_representative.set_index(pd.Series(group_by_shot.groups.keys()))

    return data_representative

### 주어진 공구의 주어진 필드에 대한 valid shot 별 대푯값 도출 & 시각화

In [22]:
#representative_value_types : 'std', 'sem', 'sum', 'mean', 'median', 'var', 'max', 'quantile(0.25)', 'quantile(0.75)', 'skew'를 리스트 형태로 입력
#  ex) ['std', 'mean']
def build_and_display_representative_data_for_tool_machining(data, tool_no, target_column_name, representative_value_types, apply_robust_scaler=False, except_min_value=False, except_max_value=False):
    df_representative = build_representative_data_for_tool_machining(data, tool_no, target_column_name, apply_robust_scaler, except_min_value, except_max_value)
    show_plot(df_representative, representative_value_types, title = target_column_name, x_axis_title='real shot no.', y_axis_title='representatives')

### 주어진 필드에 대해 Isolation Forest 기법을 적용하여 이상치 탐지

In [23]:
# he offsetis defined in such a way we obtain the expected number of outliers (samples with decision function < 0) in training.
def check_and_mark_outlier_by_IsolationForest_org(training_data, test_data, training_or_test = True, *target_column_name):
    import pickle
    from sklearn.ensemble import IsolationForest
    import joblib
    
    
    # 학습 모델    
    target_column_name = list(target_column_name)
    print(target_column_name)
    if training_or_test == True:
        data = training_data[target_column_name]
        IF = IsolationForest(random_state = 42, contamination = 0.004 , n_estimators = 500, max_samples = 90, n_jobs = -1, bootstrap = True).fit(training_data[target_column_name])
        score = IF.decision_function(data)
        training_data['IF_Outliers'] = pd.Series(IF.predict(data), index = training_data.index).apply(lambda x: 1 if x == -1 else 0)
        training_data['IF_score'] = score
        joblib.dump(IF, os.path.join(os.getcwd(), 'IF_training_model.pkl'))
        return training_data
        # training_data['score_sample'] = IF.score_samples(training_data[[target_column_name]])
        # training_data['offset'] = training_data['IF_score'] - training_data['score_sample']
#         training_data.loc[(training_data['IF_Outliers'] == 1) & (training_data['IF_score'] < 0)]['IF_Outliers'] = 0
#         training_data.loc[(training_data['IF_Outliers'] == 0) & (training_data['IF_score'] > 0)]['IF_Outliers'] = 1 
    else:
        try:
            joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
        except FileNotFoundError:
            print('IF_training_model File does not exist.')
            print('This need a learning model.')
            print('Please training model establishment first.')
        else:
            training_model = joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
            target_column_name = list(target_column_name)
            data = test_data[target_column_name]
            score = training_model.score_samples(data)
            test_data['IF_Outliers'] = pd.Series(training_model.predict(data), index = test_data.index).apply(lambda x: 1 if x == -1 else 0)
            test_data['IF_score'] = score
            return test_data
            # test_data['score_sample'] = IF.score_samples(test_data[[target_column_name]])
            # test_data['offset'] = test_data['IF_score'] - test_data['score_sample'] # offset = decision_function - sample_score
    #         test_data.loc[(test_data['IF_Outliers'] == 1) & (test_data['IF_score'] > -0.006), ['IF_Outliers']] = 0 
    #         test_data.loc[(test_data['IF_Outliers'] == 0) & (test_data['IF_score'] < -0), ['IF_Outliers']] = 1 

# 다변량 Isolation forest 처리

In [ ]:
# he offsetis defined in such a way we obtain the expected number of outliers (samples with decision function < 0) in training.
# 다변량 Isolation forest 처리
def multi_check_and_mark_outlier_by_IsolationForest_org(training_data, test_data, training_or_test = True, *target_column_name):
    import pickle
    from sklearn.ensemble import IsolationForest
    import joblib
    
    
    # 학습 모델    
    target_column_name = list(target_column_name)
    for column_idx in target_column_name:
        if column_idx not in training_data.columns:
            print("{} is not in columns\n Please input valid column name".format(column_idx))
            return
        # 데이터프레임에 결측값이 존재하는 여부를 확인처리 필요 target_column_name에 대하여
    try: 
        if training_data.isnull().sum().sum() > 0:
            raise ValueError
    except ValueError:
        print('Training data include NaN value.')
        print('First, It is essential that processing NaN value of training data.\n')
    try: 
        if test_data.isnull().sum().sum() > 0:
            raise ValueError
    except ValueError:
        print('Test data include NaN value.')
        print('First, It is essential that processing NaN value of test data.\n')

    else:
        if training_or_test == True:
            data = training_data[target_column_name]
            IF = IsolationForest(random_state = 42, contamination = 0.004 , n_estimators = 500, max_samples = 90, n_jobs = -1, bootstrap = True).fit(data)
            score = IF.decision_function(data)
            training_data['IF_Outliers'] = pd.Series(IF.predict(data), index = training_data.index).apply(lambda x: 1 if x == -1 else 0)
            training_data['IF_score'] = score
            joblib.dump(IF, os.path.join(os.getcwd(), 'IF_training_model.pkl'))
            return training_data
            # training_data['score_sample'] = IF.score_samples(training_data[[target_column_name]])
            # training_data['offset'] = training_data['IF_score'] - training_data['score_sample']
    #         training_data.loc[(training_data['IF_Outliers'] == 1) & (training_data['IF_score'] < 0)]['IF_Outliers'] = 0
    #         training_data.loc[(training_data['IF_Outliers'] == 0) & (training_data['IF_score'] > 0)]['IF_Outliers'] = 1 
        else:
            try:
                joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
            except FileNotFoundError:
                print('IF_training_model File does not exist.')
                print('This need a learning model.')
                print('Please training model establishment first.')
            else:
                training_model = joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
                target_column_name = list(target_column_name)
                data = test_data[target_column_name]
                score = training_model.score_samples(data)
                test_data['IF_Outliers'] = pd.Series(training_model.predict(data), index = test_data.index).apply(lambda x: 1 if x == -1 else 0)
                test_data['IF_score'] = score
                return test_data
                # test_data['score_sample'] = IF.score_samples(test_data[[target_column_name]])
                # test_data['offset'] = test_data['IF_score'] - test_data['score_sample'] # offset = decision_function - sample_score
        #         test_data.loc[(test_data['IF_Outliers'] == 1) & (test_data['IF_score'] > -0.006), ['IF_Outliers']] = 0 
        #         test_data.loc[(test_data['IF_Outliers'] == 0) & (test_data['IF_score'] < -0), ['IF_Outliers']] = 1 

In [24]:
# def check_and_mark_outlier_by_IsolationForest_test(test_data, target_column_name):
#     # applying test set    
#     score = check_and_mark_outlier_by_IsolationForest_org.decision_function(test_data[[target_column_name]])
#     data['IF_Outliers'] = pd.Series(check_and_mark_outlier_by_IsolationForest_org.predict(test_data[[target_column_name]])).apply(lambda x: 1 if x == -1 else 0)
#     data['IF_score'] = score
    
#     # data.loc[(data['IF_Outliers'] == 1) & (data['IF_score'] > 0), ['IF_Outliers']] = 0 #LJY_20221005 : 'IF_score' 값이 0보다 큰 경우는 '이상'으로 판별하지 않도록 처리 시도
#     # data.loc[(data['IF_Outliers'] == 0) & (data['IF_score'] < 0), ['IF_Outliers']] = 1 #LJY_20221006 : 'IF_score' 값이 0보다 작은 경우는 '정상'으로 판별하지 않도록 처리 시도

In [25]:
#dataframe에서 컬럼을 다중선택하여 데이터를 차트로 가시화
#① data : dataframe 입력
#② target_column_name : 값을 관찰할 컬럼
#③ condition_column : 조건 판별 대상 컬럼명  ex) 'state'
#④ condition_values : 일치 여부 대상 value 목록  ex) [1], [0, 1]
#⑤ dataframe에서 특정 범위만을 선택해서 차트를 가시화 하려면 index_min 또는 index_max를 지정
#   .. index_min : 디폴트값(-1)인 경우에는 데이터의 시작 row부터 포함됨
#   ..  index_max : 디폴트값(-1)인 경우에는 데이터의 끝 row까지 포함됨
#⑥ 차트 제목 표시 관련 설정
#   .. title : 차트의 제목 문자열
#   .. title_font_size : 차트의 제목을 표시할 font size
#⑦ 차트 크기 설정
#   .. figsize_horz, figsize_vert : 차트의 가로, 세로 크기
#⑧ 범례 표시 관련 설정
#   .. legend_font_size : 범례를 표시할 font size
#   .. legend_location : 범례 표시 위치 ("upper right", "lower right", "upper left", "lower left")
#⑨ 그래프 표현 관련 설정 : 선 or marker 표시
#   .. linestyle : 선 스타일의 이름을 "solid", "dashed", "dotted", "dashdot"와 같은 형식으로 입력하거나 아래를 참고하여 입력
'''
'-'  solid line style
'--' line style
'-.' dash-dot line style
':'  dotted line style
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
's'  square marker
'p'  pentagon marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
#  .. marker : 아래를 참고하여 입력 ( https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html)
'''
'.'  point marker
','  pixel marker
'o'  circle marker
'v'  triangle_down marker
'^'  triangle_up marker
'<'  triangle_left marker
'>'  triangle_right marker
'1'  tri_down marker
'2'  tri_up marker
'3'  tri_left marker
'4'  tri_right marker
'8'  octagon marker
's'  square marker
'p'  pentagon marker
'P'  plus (filled) marker
'*'  star marker
'h'  hexagon1 marker
'H'  hexagon2 marker
'+'  plus marker
'x'  x marker
'X'  x (filled) marker
'D'  diamond marker
'd'  thin_diamond marker
'|'  vline marker
'_'  hline marker
'''
def show_plot_comparing_data_by_condition(data, target_column_name, condition_column, condition_values, data_names = ('condition is true','condition is false'),
                                           index_min = -1, index_max = -1, title = None, title_font_size=20,
                                           x_axis_title = None, y_axis_title = None, axes_title_font_size=18,
                                           tick_label_font_size=15, figsize_horz=20, figsize_vert=10,
                                           legend_font_size=15, legend_location="lower right", linestyle = 'none',
                                           marker='.', marker_size=10):
    
    parameters = { 'figure.titlesize':title_font_size, 'axes.titlesize': axes_title_font_size,
                  'axes.labelsize' : tick_label_font_size, 'legend.fontsize' : legend_font_size }
    plt.rcParams.update(parameters)
    
    plt.figure(figsize = (figsize_horz, figsize_vert))
    
    condition_true_data = pd.DataFrame()
    condition_false_data = pd.DataFrame()
      
    if index_min > -1 and index_max > -1 and index_min < index_max:
        condition_true_data = data.iloc[index_min:index_max].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[index_min:index_max].query(~data[condition_column].isin(condition_values))
    elif index_min > -1:
        condition_true_data = data.iloc[index_min:].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[index_min:].query(~data[condition_column].isin(condition_values))        
    elif index_max > -1:
        condition_true_data = data.iloc[:index_max].query(data[condition_column].isin(condition_values))
        condition_false_data = data.iloc[:index_max].query(~data[condition_column].isin(condition_values))        
    else:
        condition_true_data = data[data[condition_column].isin(condition_values)]
        condition_false_data = data[~data[condition_column].isin(condition_values)]
        
    plt.plot(condition_true_data.index, condition_true_data[target_column_name], marker=marker, linestyle=linestyle, label=data_names[0], color='red', markersize=marker_size)
    plt.plot(condition_false_data.index, condition_false_data[target_column_name], marker=marker, linestyle=linestyle, label=data_names[1], markersize=marker_size)
    plt.xticks(fontsize = tick_label_font_size)
    plt.yticks(fontsize = tick_label_font_size)
    
    if title is not None:
        plt.title(title)

    plt.legend(data_names, loc = legend_location)
    
    if x_axis_title is not None:
        plt.xlabel(x_axis_title, fontsize = 15)
    if y_axis_title is not None:
        plt.ylabel(y_axis_title, fontsize = 15)

# Anomaly score 기반 threshould 도출

In [26]:
def threshould_deduction(data, target_column_name, graph = True):
    # True일 경우 training_data
    # False일 경우 data
    import matplotlib.pyplot as plt
    import pickle
    from sklearn.ensemble import IsolationForest
    import joblib
    try:
        joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
    except FileNotFoundError:
        print('IF_training_model File does not exist.')
        print('This need a learning model.')
        print('Please training model establishment first.')
    else:
        input_data = data[target_column_name].values.reshape(-1,1)
        training_model = joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
        input_data_anomaly_score = training_model.decision_function(input_data)
        input_data_outlier = training_model.predict(input_data)
        temp_input = np.concatenate((input_data.reshape(-1,1), input_data_outlier.reshape(-1,1)), axis = 1)
        result = pd.DataFrame(temp_input, columns = [target_column_name, 'outlier'])
        if graph == True:
            plt.figure(figsize = (20,12))
            plt.scatter(input_data, input_data_anomaly_score, label = 'anomaly score')
            plt.fill_between(input_data.T[0], np.min(input_data_anomaly_score), np.max(input_data_anomaly_score), where=input_data_outlier==-1, color='r', 
                             alpha=.3, label='outlier region')
            plt.axvline(min(result.query('outlier == -1')[target_column_name]),  color = 'g', linestyle = 'dashed')
            plt.text(min(result.query('outlier == -1')[target_column_name]), np.min(input_data_anomaly_score)-0.1, 'Threshold {:.4f}'.format(min(result.query('outlier == -1')[target_column_name])), color = 'r', fontsize = 12, rotation = 90)
            plt.legend()
            plt.ylabel('anomaly score', fontsize = 15)
            plt.yticks(fontsize = 13)
            plt.xlabel(target_column_name, fontsize = 15)
            plt.xticks(fontsize = 13)
        return min(result.query('outlier == -1')[target_column_name])

In [ ]:
def multi_threshould_deduction(data, graph = True, *target_column_name):
    # True일 경우 training_data
    # False일 경우 data
    import matplotlib.pyplot as plt
    import pickle
    from sklearn.ensemble import IsolationForest
    import joblib
    try:
        joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
    except FileNotFoundError:
        print('IF_training_model File does not exist.')
        print('This need a learning model.')
        print('Please training model establishment first.')
    else:
        target_column_name = list(target_column_name)
        training_model = joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
        if len(target_column_name) == 1:
            input_data = data[target_column_name].values.reshape(-1,1)
            input_data_anomaly_score = training_model.decision_function(input_data)
            input_data_outlier = training_model.predict(input_data)
            temp_input = np.concatenate((input_data.reshape(-1,1), input_data_outlier.reshape(-1,1)), axis = 1)
            result = pd.DataFrame(temp_input, columns = [target_column_name, 'outlier'])
        else: 
            input_data = data[target_column_name].values
            input_data_anomaly_score = training_model.decision_function(input_data)
            input_data_outlier = training_model.predict(input_data)
            result = data[target_column_name]
            result['outlier'] = input_data_outlier
        #return min(result.query('outlier == -1')[target_column_name])
        
        if graph == True:
            fig = plt.figure(figsize = (20,12))
            if len(target_column_name) == 1:
                plt.figure(figsize = (20,12))
                plt.scatter(input_data.T[idx], input_data_anomaly_score, label = 'anomaly score')
                plt.fill_between(input_data.T[idx], np.min(input_data_anomaly_score), np.max(input_data_anomaly_score), where=input_data_outlier==-1, color='r', 
                                 alpha=.3, label='outlier region')
                plt.axvline(min(result.query('outlier == -1')[target_column_name]),  color = 'g', linestyle = 'dashed')
                plt.text(min(result.query('outlier == -1')[target_column_name]), np.min(input_data_anomaly_score)-0.2, 'Threshold {:.4f}'.format(min(result.query('outlier == -1')[target_column_name])), color = 'r', fontsize = 12, rotation = 90)
                plt.legend()
                plt.ylabel('anomaly score', fontsize = 15)
                plt.yticks(fontsize = 13)
                plt.xlabel(target_column_name, fontsize = 15)
                plt.xticks(fontsize = 13)
                
            else:
    
                for idx in range(len(target_column_name)):
                    globals()['ax_{}'.format(idx)] = fig.add_subplot(len(target_column_name), 1, idx+1)
                    globals()['ax_{}'.format(idx)].scatter(input_data.T[idx], input_data_anomaly_score, label = 'anomaly score')
                    globals()['ax_{}'.format(idx)].fill_between(input_data.T[idx], np.min(input_data_anomaly_score), np.max(input_data_anomaly_score), where=input_data_outlier==-1, color='r', 
                                     alpha=.5, label='outlier region')
                    print(result.query('outlier == -1'))
                    globals()['ax_{}'.format(idx)].axvline(min(result.query('outlier == -1')[target_column_name[idx]]),  color = 'g', linestyle = 'dashed')
                    globals()['ax_{}'.format(idx)].text(min(result.query('outlier == -1')[target_column_name[idx]]), np.min(input_data_anomaly_score)-0.08, 'Threshold', fontsize = 12, color = 'g')
                    # plt.legend()
                    globals()['ax_{}'.format(idx)].set_ylabel('anomaly score', fontsize = 15)
                    #plt.yticks(fontsize = 13)
                    globals()['ax_{}'.format(idx)].set_xlabel('{}'.format(target_column_name[idx]), fontsize = 15)
                    
            return min(result.query('outlier == -1')[target_column_name])

                    #plt.xticks(fontsize = 13)
    

# Threshould를 적용한 시각화 적용

In [27]:
def show_plot_with_threshould(data, target_column_name, condition_column, condition_values, data_names = ('condition is true','condition is false'),
                                           index_min = -1, index_max = -1, title = None, title_font_size=20,
                                           x_axis_title = None, y_axis_title = None, axes_title_font_size=18,
                                           tick_label_font_size=15, figsize_horz=20, figsize_vert=10,
                                           legend_font_size=15, legend_location="lower right", linestyle = 'none',
                                           marker='.', marker_size=10):
    
    try:
        joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
    except FileNotFoundError:
        print('IF_training_model File does not exist.')
        print('This need a learning model.')
        print('Please training model establishment first.')
    else:
        parameters = { 'figure.titlesize':title_font_size, 'axes.titlesize': axes_title_font_size,
                      'axes.labelsize' : tick_label_font_size, 'legend.fontsize' : legend_font_size }
        plt.rcParams.update(parameters)

        plt.figure(figsize = (figsize_horz, figsize_vert))

        input_data = data[target_column_name].values.reshape(-1,1)
        training_model = joblib.load(os.path.join(os.getcwd(), 'IF_training_model.pkl'))
        input_data_anomaly_score = training_model.decision_function(input_data)
        input_data_outlier = training_model.predict(input_data)
        temp_input = np.concatenate((input_data.reshape(-1,1), input_data_outlier.reshape(-1,1)), axis = 1)

        condition_true_data = pd.DataFrame()
        condition_false_data = pd.DataFrame()

        if index_min > -1 and index_max > -1 and index_min < index_max:
            condition_true_data = data.iloc[index_min:index_max].query(data[condition_column].isin(condition_values))
            condition_false_data = data.iloc[index_min:index_max].query(~data[condition_column].isin(condition_values))
        elif index_min > -1:
            condition_true_data = data.iloc[index_min:].query(data[condition_column].isin(condition_values))
            condition_false_data = data.iloc[index_min:].query(~data[condition_column].isin(condition_values))        
        elif index_max > -1:
            condition_true_data = data.iloc[:index_max].query(data[condition_column].isin(condition_values))
            condition_false_data = data.iloc[:index_max].query(~data[condition_column].isin(condition_values))        
        else:
            condition_true_data = data[data[condition_column].isin(condition_values)]
            condition_false_data = data[~data[condition_column].isin(condition_values)]

        plt.plot(condition_true_data.index, condition_true_data[target_column_name], marker=marker, linestyle=linestyle, label=data_names[0], color='red', markersize=marker_size)
        plt.plot(condition_false_data.index, condition_false_data[target_column_name], marker=marker, linestyle=linestyle, label=data_names[1], markersize=marker_size)
        plt.axhline(threshould_deduction(data, target_column_name, graph = False), color = 'r', linestyle = 'dashed')
        plt.text(-290, threshould_deduction(data, target_column_name, graph = False)-15, 'Threshold {:.4f}'.format(min(pd.DataFrame(temp_input, columns = [target_column_name, 'outlier']).query('outlier == -1')[target_column_name])), color = 'r', fontsize = 12)

        if title is not None:
            plt.title(title)

        plt.legend(data_names, loc = legend_location)
        plt.xticks(fontsize = 15)
        plt.yticks(fontsize = 15)

        if x_axis_title is not None:
            plt.xlabel(x_axis_title, fontsize = 15)
        if y_axis_title is not None:
            plt.ylabel(y_axis_title, fontsize = 15)